In [1]:
import base64
import hmac
import json
import time
import requests
import toml

from urllib.parse import urljoin

# params
base_url = 'https://www.dojqutmbiuuo.com'
api_config = toml.load('../config/demo_api.toml')
api_key = api_config['api_key']
secret_key = api_config['secret_key']
passphrase = api_config['passphrase']

In [2]:
def request(method, uri, params=None, body=None, headers=None, auth=False, demo=False):
    """Initiate network request
   @param method: request method, GET / POST / DELETE / PUT
   @param uri: request uri
   @param params: dict, request query params
   @param body: dict, request body
   @param headers: request http header
   @param auth: boolean, add permission verification or not
   """
    if params:
        query = "&".join(
            ["{}={}".format(k, params[k]) for k in sorted(params.keys())]
        )
        uri += "?" + query
    url = urljoin(base_url, uri)

    if auth:
        timestamp = (
                str(time.time()).split(".")[0]
                + "."
                + str(time.time()).split(".")[1][:3]
        )
        if body:
            body = json.dumps(body)
        else:
            body = ""
        message = str(timestamp) + str.upper(method) + uri + str(body)
        mac = hmac.new(
            bytes(secret_key, encoding="utf8"),
            bytes(message, encoding="utf-8"),
            digestmod="sha256",
        )
        d = mac.digest()
        sign = base64.b64encode(d)

        if not headers:
            headers = {}
        headers["Content-Type"] = "application/json"
        headers["OK-ACCESS-KEY"] = api_key
        headers["OK-ACCESS-SIGN"] = sign
        headers["OK-ACCESS-TIMESTAMP"] = str(timestamp)
        headers["OK-ACCESS-PASSPHRASE"] = passphrase
        if demo:
            headers["x-simulated-trading"] = "1"
    result = requests.request(
        method, url, data=body, headers=headers, timeout=10
    ).json()
    if result.get("code") and result.get("code") != "0":
        return None, result
    return result, None

In [3]:
def get_asset(currency):
    params = {"ccy": currency}
    result = request(
        "GET", "/api/v5/account/balance", params=params, auth=True, demo=True
    )
    return result


get_asset('USDT')

({'code': '0',
  'data': [{'adjEq': '',
    'borrowFroz': '',
    'details': [{'accAvgPx': '',
      'availBal': '57065.91641669225',
      'availEq': '57065.91641669225',
      'borrowFroz': '',
      'cashBal': '57065.91641669225',
      'ccy': 'USDT',
      'clSpotInUseAmt': '',
      'crossLiab': '',
      'disEq': '57044.231368453904',
      'eq': '57065.91641669225',
      'eqUsd': '57044.231368453904',
      'fixedBal': '0',
      'frozenBal': '0',
      'imr': '0',
      'interest': '',
      'isoEq': '0',
      'isoLiab': '',
      'isoUpl': '0',
      'liab': '',
      'maxLoan': '',
      'maxSpotInUse': '',
      'mgnRatio': '',
      'mmr': '0',
      'notionalLever': '0',
      'openAvgPx': '',
      'ordFrozen': '0',
      'rewardBal': '0',
      'smtSyncEq': '0',
      'spotBal': '',
      'spotCopyTradingEq': '0',
      'spotInUseAmt': '',
      'spotIsoBal': '0',
      'spotUpl': '',
      'spotUplRatio': '',
      'stgyEq': '0',
      'totalPnl': '',
      'totalPnlR

In [4]:
def trade_spot(spot_id, side, order_type, order_price, order_vol, demo=False):
	params = {
        "instId": spot_id,
        "tdMode": 'cash',
        "side": side,
        "ordType": order_type,
		"px": f"{order_price:.10f}",
        "sz": f"{order_vol:.10f}"
	}

	resp = request("POST", "/api/v5/trade/order", body=params, auth=True, demo=demo)
	return resp

def open_position_future(future_id, side, order_type, order_price, order_vol,
						  margin_mode='isolated', demo=False, order_id=""):
	params = {
        "instId": future_id,
        "tdMode": margin_mode,
        "side": side,
        "ordType": order_type,
		"px": f"{order_price:.10f}",
        "sz": f"{order_vol:.10f}",
	}

	if order_id != "":
		params["clOrdId"] = order_id

	resp = request("POST", "/api/v5/trade/order", body=params, auth=True, demo=demo)
	return resp

def set_future_lever(future_id, lever, margin_mode, demo=False):
	params = {
		"instId": future_id,
		"lever": str(lever),
		"mgnMode": margin_mode
	}

	resp = request("POST", "/api/v5/account/set-leverage", body=params, auth=True, demo=demo)
	return resp

	
def close_position_future(future_id, margin_mode, demo=False, order_id=""):
	params = {
        "instId": future_id,
        "mgnMode": margin_mode,
	}

	if order_id != "":
		params["clOrdId"] = order_id

	resp = request("POST", "/api/v5/trade/close-position", body=params, auth=True, demo=demo)
	return resp

def query_position(future_id, demo=False):
	params = {
		"instId": future_id
	}

	resp = request("GET", "/api/v5/account/positions", params=params, auth=True, demo=demo)
	return resp

In [5]:
resp = set_future_lever("BTC-USDT-SWAP", 3, "isolated", demo=True)
resp

({'code': '0',
  'data': [{'instId': 'BTC-USDT-SWAP',
    'lever': '3',
    'mgnMode': 'isolated',
    'posSide': ''}],
  'msg': ''},
 None)

In [11]:
import time
start_time = time.time()
resp = open_position_future("BTC-USDT-SWAP", "buy", "market", 0.0, 0.1, demo=True, order_id='20241215btc')
end_time = time.time()
execution_time = end_time - start_time
print(f"time cost = {execution_time:.4f} sec")

time cost = 0.1952 sec


In [70]:
resp = query_position("BTC-USDT-SWAP", demo=True)
resp

({'code': '0',
  'data': [{'adl': '1',
    'availPos': '',
    'avgPx': '102031.1',
    'baseBal': '',
    'baseBorrowed': '',
    'baseInterest': '',
    'bePx': '102133.18214107053',
    'bizRefId': '',
    'bizRefType': '',
    'cTime': '1734244097308',
    'ccy': 'USDT',
    'clSpotInUseAmt': '',
    'closeOrderAlgo': [],
    'deltaBS': '',
    'deltaPA': '',
    'fee': '-0.05101555',
    'fundingFee': '0',
    'gammaBS': '',
    'gammaPA': '',
    'idxPx': '102034.2000000000000000',
    'imr': '',
    'instId': 'BTC-USDT-SWAP',
    'instType': 'SWAP',
    'interest': '',
    'last': '102028',
    'lever': '10',
    'liab': '',
    'liabCcy': '',
    'liqPenalty': '0',
    'liqPx': '92243.18387744852',
    'margin': '10.20311',
    'markPx': '102035.9',
    'maxSpotInUseAmt': '',
    'mgnMode': 'isolated',
    'mgnRatio': '22.231630675521476',
    'mmr': '0.4081435999999999',
    'notionalUsd': '102.00222815299999',
    'optVal': '',
    'pendingCloseOrdLiabVal': '',
    'pnl': '0'

In [12]:
resp = close_position_future("BTC-USDT-SWAP", "isolated", demo=True)
resp

({'code': '0',
  'data': [{'clOrdId': '',
    'instId': 'BTC-USDT-SWAP',
    'posSide': 'net',
    'tag': ''}],
  'msg': ''},
 None)

In [21]:
resp = trade_spot("BTC-USDT", "buy", "market", 0.0, 2)  # buy 2 usdt btc at market price
resp

(None, {'msg': 'Invalid Sign', 'code': '50113'})

In [45]:
resp = trade_spot("BTC-USDT", "sell", "market", 0.0, 0.00001) # sell 0.00001 btc at market price 
resp

({'code': '0',
  'data': [{'clOrdId': '',
    'ordId': '2069818259946872832',
    'sCode': '0',
    'sMsg': 'Order placed',
    'tag': '',
    'ts': '1734187791066'}],
  'inTime': '1734187791065936',
  'msg': '',
  'outTime': '1734187791067773'},
 None)